In [1]:
import sys

sys.path.append('../script')
import get_dataset as ds

In [2]:
feats, dialects = ds.load_windowed_dataset('mel_spectrogram', split=False, onehot=False, normalised=True)

Using normalisation method: True
Loading Ngadas: 
- Marsam.wav (time: 0.9)
- Mas_Tom_Ngadas_201102.wav (time: 0.0)
Loading Banten: 
- JWBP-20121229-A.wav (time: 0.2)
- JWBA-20100703-a.wav (time: 0.2)
- JWB-Banten_9.wav (time: 0.4)
- banten-2010-07-02-a.wav (time: 0.2)
- JWB-20090125.wav (time: 0.2)
- JWBP-20121229-B.wav (time: 0.0)
- JWB-Banten_10.wav (time: 0.1)
- JWB-20090126.wav (time: 0.3)
- JWB-20090116.wav (time: 0.1)
- JWB-Banten20100605-B.wav (time: 0.0)
- JWB-Banten_1.wav (time: 0.0)
- JWB-Banten_6.wav (time: 0.1)
- JWBA-20100703-b.wav (time: 0.1)
- JWB-Banten_4.wav (time: 0.0)
- JWB-20090117.wav (time: 0.1)
- JWB-20090118.wav (time: 0.1)
- JWB-Banten_8.wav (time: 0.0)
- JWB-20090131.wav (time: 0.5)
- JWB-20090130.wav (time: 0.2)
- JWB-Banten_11.wav (time: 0.0)
- JWB-Banten_5.wav (time: 0.0)
- JWB-20090123.wav (time: 0.1)
- JWB-20090124.wav (time: 0.1)
- JWB-20090127.wav (time: 0.2)
- JWB-Banten_19.wav (time: 0.1)
- JWB-Banten20100605-A.wav (time: 0.2)
- JWB-Banten_7.wav (time

In [3]:
feats.squeeze().shape

(41594, 128, 150)

In [4]:
import tensorflow as tf

feats_ds = tf.data.Dataset.from_tensor_slices((feats.squeeze(axis=-1), dialects.reshape(-1)))
feats_ds

2022-02-08 21:24:08.254370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 21:24:08.305670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 21:24:08.305897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 21:24:08.306510: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

<TensorSliceDataset shapes: (128, 150), types: tf.float32>

In [5]:
dialects_ds

<TensorSliceDataset shapes: (), types: tf.string>

In [6]:
for feat, dial in feats_ds.take(1):
    tf_feat = feat
    tf_dial = dial
    print(feat, dial, sep='\n')

2022-02-08 21:24:32.660786: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3194419200 exceeds 10% of free system memory.


tf.Tensor(
[[5.56965896e-09 6.56368471e-09 1.80016970e-03 ... 2.20611900e-01
  5.15196383e-01 7.38890767e-01]
 [8.17684920e-10 9.63618407e-10 2.64283997e-04 ... 3.23881656e-02
  7.56362900e-02 1.08476996e-01]
 [1.28410260e-09 3.69706799e-09 2.02417257e-03 ... 2.57737994e+00
  5.48182309e-01 1.18291564e-01]
 ...
 [1.59491465e-11 6.07183531e-12 1.91526979e-05 ... 2.47445935e-03
  4.28777828e-04 3.42490594e-03]
 [1.26028517e-12 6.92756508e-13 4.54094425e-06 ... 1.42068791e-04
  1.40367309e-04 2.53594451e-04]
 [1.89607681e-14 1.05956613e-14 1.75300073e-07 ... 7.25279779e-06
  1.31245861e-05 5.79931839e-06]], shape=(128, 150), dtype=float32)


In [7]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(feats, dialects):
    feature = {
        'feature': _bytes_feature(tf.io.serialize_tensor(feats).numpy()),
        'dialect': _bytes_feature(dialects)
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def tf_serialize_example(feats, dialects):
    tf_string = tf.py_function(
        serialize_example,
        (feats, dialects),
        tf.string
    )
    return tf.reshape(tf_string, ())

In [9]:
serialized_feature_dataset = feats_ds.map(tf_serialize_example)
serialized_feature_dataset

<MapDataset shapes: (), types: tf.string>

In [11]:
def generator():
    for features in feats_ds:
        yield serialize_example(*features)

In [12]:
serialized_feature_dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=tf.string,
    output_shapes=()
)

In [14]:
filename = '../dataset/tfrecords/mel_spectrogram_windowed_normalised.tfrecords'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_feature_dataset)